In [34]:
#Imports
import os, os.path
from whoosh import index
from whoosh import fields
from whoosh.analysis import StemmingAnalyzer
from whoosh.analysis import StandardAnalyzer
from whoosh.analysis import StopFilter
from whoosh.lang.porter import stem
import glob
import codecs
from whoosh import scoring

In [35]:
import atexit
from time import time, strftime, localtime
from datetime import timedelta

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%Y-%m-%d %H:%M:%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    print(line)
    print(secondsToStr(), '-', s)
    if elapsed:
        print("Elapsed time:", elapsed)
    print(line)
    print()

def endlog():
    end = time()
    elapsed = end-start
    log("End Program", secondsToStr(elapsed))

start = time()
atexit.register(endlog)
log("Start Program")
endlog()

2018-09-19 11:37:01 - Start Program

2018-09-19 11:37:01 - End Program
Elapsed time: 0:00:00



In [39]:
#SEM STEMMING/STOPWORDS
if not os.path.exists("stan"):
    os.mkdir("stan")
    
stan_ana = StandardAnalyzer(stoplist=None)
schema_stan = fields.Schema(title=fields.TEXT(analyzer=stan_ana, stored=True), content=fields.TEXT(analyzer=stan_ana))

ix_stan = index.create_in("stan", schema_stan)

TypeError: StandardAnalyzer() got an unexpected keyword argument 'commas'

In [3]:
#COM STEMMING
if not os.path.exists("stem"):
    os.mkdir("stem")
    
stem_ana = StemmingAnalyzer(stoplist=None, minsize=2, maxsize=None, gaps=False, stemfn=stem, ignore=None, cachesize=50000)
schema_stem = fields.Schema(title=fields.TEXT(analyzer=stem_ana, stored=True), content=fields.TEXT(analyzer=stem_ana))

ix_stem = index.create_in("stem", schema_stem)

In [4]:
#COM STOPWORDS
if not os.path.exists("stop"):
    os.mkdir("stop")
    
stop_ana = StandardAnalyzer(stoplist=frozenset(['and', 'is', 'it', 'an', 'as', 'at', 'have', 'in', 'yet', 'if', 'from', 'for', 'when', 'by', 'to', 'you', 'be', 'we', 'that', 'may', 'not', 'with', 'tbd', 'a', 'on', 'your', 'this', 'of', 'us', 'will', 'can', 'the', 'or', 'are']))
schema_stop = fields.Schema(title=fields.TEXT(analyzer=stop_ana, stored=True), content=fields.TEXT(analyzer=stop_ana))

ix_stop = index.create_in("stop", schema_stop)

In [5]:
#COM STEMMING E STOPWORDS
if not os.path.exists("all"):
    os.mkdir("all")
    
all_ana = StemmingAnalyzer(stoplist=frozenset(['and', 'is', 'it', 'an', 'as', 'at', 'have', 'in', 'yet', 'if', 'from', 'for', 'when', 'by', 'to', 'you', 'be', 'we', 'that', 'may', 'not', 'with', 'tbd', 'a', 'on', 'your', 'this', 'of', 'us', 'will', 'can', 'the', 'or', 'are']), minsize=2, maxsize=None, gaps=False, stemfn=stem, ignore=None, cachesize=50000)
schema_all = fields.Schema(title=fields.TEXT(analyzer=all_ana, stored=True), content=fields.TEXT(analyzer=all_ana))

ix_all = index.create_in("all", schema_all)

In [6]:
#Leitura dos arquivos

ix_stan = index.open_dir("./stan")
ix_stem = index.open_dir("./stem")
ix_stop = index.open_dir("./stop")
ix_all = index.open_dir("./all")

w_stem = ix_stem.writer(procs=4, multisegment=True,limitmb=512)
w_stan = ix_stan.writer(procs=4, multisegment=True,limitmb=512)
w_stop = ix_stop.writer(procs=4, multisegment=True,limitmb=512)
w_all = ix_all.writer(procs=4, multisegment=True,limitmb=512)

start = time()
atexit.register(endlog)
log("Leitura")

count = 0;
for path in glob.glob("./docs/*.pdf"):
    path = path.replace("\\","/")
    with open(path, "r",encoding="utf-8", errors='ignore') as f:
        content = f.read()
        if count%10==0:
            print(count)
        count+=1
        w_stem.update_document(title=os.path.basename(path), content=content)
        w_stan.update_document(title=os.path.basename(path), content=content)
        w_stop.update_document(title=os.path.basename(path), content=content)
        w_all.update_document(title=os.path.basename(path), content=content)
        f.close()
endlog()
print("Commitando Stemming")
start = time()
atexit.register(endlog)
log("Standard")
w_stem.commit()
endlog()

start = time()
atexit.register(endlog)
log("Stemming")
#print("Commitando Standard")
w_stan.commit()
endlog()

start = time()
atexit.register(endlog)
log("Stopword")
#print("Commitando Stopwords")
w_stop.commit()
endlog()

start = time()
atexit.register(endlog)
log("Stopword and Stemming")
#print("Commitando Stemming+Stopwords")
w_all.commit()
endlog()

LENDO ARQUIVOS...
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
LEITURA E ADIÇÃO COMPLETAS!

Commitando Stemming
Commitando Standard
Commitando Stopwords
Commitando Stemming+Stopwords


In [33]:
from whoosh.qparser import QueryParser

search = "machine learning applications"
start = time()
atexit.register(endlog)
log("Start Program")

with ix_stan.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stan.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STANDARD: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

with ix_stem.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stem.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STEM: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

with ix_stop.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stop.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STOP: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())
        
with ix_all.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_all.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS ALL: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

endlog()

2018-09-19 11:30:09 - Start Program

RESULTADOS STANDARD:  43
<Hit {'title': 'a292607.pdf'}>
[('content', b'learning'), ('content', b'machine'), ('content', b'applications')]
<Hit {'title': 'State-of-the-Art Deep Learning Evolving Machine Intelligence Toward Tomorrow-s Intelligent Network Traffic Control Systems.pdf'}>
[('content', b'learning'), ('content', b'machine'), ('content', b'applications')]
<Hit {'title': 'Classification of hyperspectral images with multiple kernel extreme learning machine.pdf'}>
[('content', b'learning'), ('content', b'machine'), ('content', b'applications')]
<Hit {'title': 'A Machine Learning Based WSN System for Autism Activity Recognition.pdf'}>
[('content', b'learning'), ('content', b'machine'), ('content', b'applications')]
<Hit {'title': 'Increasing system performance in machine learning by using multiprocessing.pdf'}>
[('content', b'learning'), ('content', b'machine'), ('content', b'applications')]
<Hit {'title': 'Mobile application review classificati

In [30]:
from whoosh.qparser import QueryParser

search = "personalized OR healthcare big data"

with ix_stan.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stan.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STANDARD: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

with ix_stem.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stem.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STEM: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

with ix_stop.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_stop.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS STOP: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())
        
with ix_all.searcher(weighting=scoring.TF_IDF()) as searcher:
    query = QueryParser("content", ix_all.schema).parse(search)
    results = searcher.search(query, optimize=True, scored=True, terms=True)
    print("RESULTADOS ALL: ", len(results))
    for result in results:
        print(result)
        print(result.matched_terms())

RESULTADOS STANDARD:  16
<Hit {'title': 'BigDataSurvey2014.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': '1-s2.0-S1532046413001007-main.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': '1-s2.0-S0040162516000500-main.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': 'Big-data-analytics--Understanding-its-capabilities_2018_Technological-Foreca.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': 'MONET-SmartClothing-2016.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': 'Research on Application of Healthcare Data in Big Data Era.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': '1-s2.0-S1877050915005220-main.pdf'}>
[('content', b'big'), ('content', b'data'), ('content', b'healthcare')]
<Hit {'title': 'Big Data Security and Pri

In [64]:
import pandas as pd
mlappList = []
bdfhList = []
mlappDict

for path in glob.glob("./mlapp/*.pdf"):
    mlappDict = {"Documento":os.path.basename(path),"Consulta 1":"1", "Consulta 2": "0"}
    mlappList.append(mlappDict)
for path in glob.glob("./bdfh/*.pdf"):
    bdfhDict = {"Documento":os.path.basename(path),"Consulta 1":"0", "Consulta 2": "1"}
    bdfhList.append(bdfhDict)
for item in mlappList:
    bdfhList.append(item)
#display(bdfhList)

df = pd.DataFrame.from_dict(bdfhList)
display(df)
df.to_csv("tabela.csv")
#display(mlappList)


,Consulta 1,Consulta 2,Documento
0,0,1,1-s2.0-S0040162516000500-main.pdf
1,0,1,1-s2.0-S0268401214001066-main.pdf
2,0,1,1-s2.0-S1532046413001007-main.pdf
3,0,1,1-s2.0-S1877050915005220-main.pdf
4,0,1,1-s2.0-S2212827114000857-main.pdf
5,0,1,10.1.1.452.9557.pdf
6,0,1,10334-ar-promise-peril-of-big-data.pdf
7,0,1,11606_2013_Article_2455.pdf
8,0,1,2196-1115-1-2.pdf
9,0,1,2_The_New_York_Times_on_The_Age_of_Big_D.pdf
